# AOI 模型訓練/部署/驗證

1. [引言](#引言)
2. [初始化環境](#初始化環境)
3. [設定資料](#設定資料)
4. [設定模型與其參數](#設定模型與其參數)
5. [執行模型訓練工作](#執行模型訓練工作)
6. [部署模型](#部署模型)
7. [驗證模型](#驗證模型)

## 引言

準備好資料後，我們將透過此筆記本訓練模型、部署模型、驗證模型。流程包含：
1. 初始化環境
2. 設定資料
3. 設定模型與其參數
4. 執行模型訓練工作
5. 部署模型
6. 驗證模型

在此使用 SageMaker 內建的模型與官方建議的參數，如想進一步提升模型準確度，常見的方式包含選擇更強大的模型和調整模型參數，或考慮給模型更多更好的訓練資料。

## 初始化環境

In [ ]:
# set up and authenticate the use of AWS services

import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
sess = sagemaker.Session()

In [ ]:
# get Amazon SageMaker Object Detection docker image, which is static and need not be changed.

from sagemaker.amazon.amazon_estimator import get_image_uri

training_image = get_image_uri(sess.boto_region_name, 'object-detection', repo_version="latest")
print (training_image)

## 資料設定

In [ ]:
# S3 bucket name and prefix

bucket = 'sagemaker-aoi-xxx'
prefix = 'DEMO-AOI'

In [ ]:
# input location at S3
train_channel = prefix + '/train'
validation_channel = prefix + '/validation'

s3_train_data = 's3://{}/{}'.format(bucket, train_channel)
s3_validation_data = 's3://{}/{}'.format(bucket, validation_channel)

In [ ]:
# output location at S3, where the model artifact will be dumped

s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

In [ ]:
# data channels for algorithms to consume

train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='application/x-recordio', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3_validation_data, distribution='FullyReplicated', 
                             content_type='application/x-recordio', s3_data_type='S3Prefix')
data_channels = {'train': train_data, 'validation': validation_data}

## 設定模型與其參數
如想調整模型和參數，請參閱[官方文件](https://docs.aws.amazon.com/sagemaker/latest/dg/object-detection.html)。

In [ ]:
#create a sageMaker.estimator.Estimator object. This estimator will launch the training job.
od_model = sagemaker.estimator.Estimator(training_image,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.p3.2xlarge',
                                         train_volume_size = 10,
                                         train_max_run = 36000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

In [ ]:
od_model.set_hyperparameters(base_network='resnet-50',
                             use_pretrained_model=1,
                             num_classes=6,
                             mini_batch_size=32,
                             epochs=1000,
                             learning_rate=0.001,
                             lr_scheduler_step='3,6',
                             lr_scheduler_factor=0.1,
                             optimizer='sgd',
                             momentum=0.9,
                             weight_decay=0.0005,
                             overlap_threshold=0.5,
                             nms_threshold=0.45,
                             image_shape=300,
                             label_width=60,
                             num_training_samples=1500)

### 自動調整模型參數 (可跳過)

In [ ]:
# from sagemaker.tuner import HyperparameterTuner, CategoricalParameter, ContinuousParameter, IntegerParameter

# # Configure HyperparameterTuner
# hyper_ranges = {'mini_batch_size': IntegerParameter(4, 8),
#                 'learning_rate': ContinuousParameter(0.000125, 0.00025)}
# my_tuner = HyperparameterTuner(estimator=od_model,
#                                objective_metric_name='validation:mAP',
#                                hyperparameter_ranges=hyper_ranges,
#                                max_jobs=10,
#                                max_parallel_jobs=2,
#                                early_stopping_type='Auto')
# my_tuner.fit(inputs=data_channels, include_cls_metadata=False, logs=True)
# my_tuner.wait()

# # Retrieve analytics object
# my_tuner_analytics = my_tuner.analytics()

# # Look at summary of associated training jobs
# my_dataframe = my_tuner_analytics.dataframe()

# # Plot results on graph
# import bokeh
# import bokeh.io
# bokeh.io.output_notebook()
# from bokeh.plotting import figure, show
# from bokeh.models import HoverTool

# class HoverHelper():

#     def __init__(self, tuning_analytics):
#         self.tuner = tuning_analytics

#     def hovertool(self):
#         tooltips = [
#             ("FinalObjectiveValue", "@FinalObjectiveValue"),
#             ("TrainingJobName", "@TrainingJobName"),
#         ]
#         for k in self.tuner.tuning_ranges.keys():
#             tooltips.append( (k, "@{%s}" % k) )

#         ht = HoverTool(tooltips=tooltips)
#         return ht

#     def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
#         return [self.hovertool(), standard_tools]

# hover = HoverHelper(my_tuner_analytics)

# p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='datetime')
# p.circle(source=my_dataframe, x='TrainingStartTime', y='FinalObjectiveValue')
# show(p)

# # Plot correlation on graph
# objective_name = my_tuner.objective_metric_name

# ranges = tuner.tuning_ranges
# figures = []
# for hp_name, hp_range in ranges.items():
#     categorical_args = {}
#     if hp_range.get('Values'):
#         # This is marked as categorical.  Check if all options are actually numbers.
#         def is_num(x):
#             try:
#                 float(x)
#                 return 1
#             except:
#                 return 0           
#         vals = hp_range['Values']
#         if sum([is_num(x) for x in vals]) == len(vals):
#             # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
#             print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
#         else:
#             # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
#             categorical_args['x_range'] = vals

#     # Now plot it
#     p = figure(plot_width=500, plot_height=500, 
#                title="Objective vs %s" % hp_name,
#                tools=hover.tools(),
#                x_axis_label=hp_name, y_axis_label=objective_name,
#                **categorical_args)
#     p.circle(source=df, x=hp_name, y='FinalObjectiveValue')
#     figures.append(p)
# show(bokeh.layouts.Column(*figures))

# # deploy best model
# my_predictor = my_tuner.deploy(initial_instance_count = 1, instance_type = 'ml.t2.medium')

## 執行模型訓練工作
執行以下程式 `.fit()` 將開始訓練工作，以上述參數為例，執行時間約 3 小時。過程中會持續顯示 Mean Average Prceision (mAP) 以評估模型好壞，模型隨時訓練時間愈長，準確度愈高。

訓練工作執行完成後，會顯示 "Training job completed" 的訊息，並在上述指定的 S3 路徑存有訓練好的模型。

In [ ]:
od_model.fit(inputs=data_channels, logs=True)

## 部署模型

In [ ]:
object_detector = od_model.deploy(initial_instance_count = 1, instance_type = 'ml.t2.medium')

## 驗證模型
下述程式會使用一圖片測試模型，並將辨識結果畫出來。

In [ ]:
%matplotlib inline

def visualize_detection(img_file, dets, classes=[], thresh=0.6):
        """
        visualize detections in one image
        Parameters:
        ----------
        img : numpy.array
            image, in bgr format
        dets : numpy.array
            ssd detections, numpy.array([[id, score, x1, y1, x2, y2]...])
            each row is one object
        classes : tuple or list of str
            class names
        thresh : float
            score threshold
        """
        import random
        import matplotlib.pyplot as plt
        import matplotlib.image as mpimg

        img=mpimg.imread(img_file)
        plt.imshow(img)
        height = img.shape[0]
        width = img.shape[1]
        colors = dict()
        for det in dets:
            (klass, score, x0, y0, x1, y1) = det
            if score < thresh:
                continue
            cls_id = int(klass)
            if cls_id not in colors:
                colors[cls_id] = (random.random(), random.random(), random.random())
            xmin = int(x0 * width)
            ymin = int(y0 * height)
            xmax = int(x1 * width)
            ymax = int(y1 * height)
            rect = plt.Rectangle((xmin, ymin), xmax - xmin,
                                 ymax - ymin, fill=False,
                                 edgecolor=colors[cls_id],
                                 linewidth=3.5)
            plt.gca().add_patch(rect)
            class_name = str(cls_id)
            if classes and len(classes) > cls_id:
                class_name = classes[cls_id]
            plt.gca().text(xmin, ymin - 2,
                            '{:s} {:.3f}'.format(class_name, score),
                            bbox=dict(facecolor=colors[cls_id], alpha=0.5),
                                    fontsize=12, color='white')
        plt.show()

In [ ]:
file_name = 'NEU-DET/IMAGES/scratches_55.jpg'

with open(file_name, 'rb') as image:
    f = image.read()
    b = bytearray(f)
    ne = open('n.txt','wb')
    ne.write(b)

In [ ]:
%%time

import json

object_detector.content_type = 'image/jpeg'
results = object_detector.predict(b)
detections = json.loads(results)
print (detections)

In [ ]:
object_categories = ['黑斑', '介在物', '孔蝕', '軋入鏽皮', '刮痕']

# setting a threshold 0.20 will only plot detection results that have a confidence score greater than 0.20
threshold = 0.20

# visualize the detections
visualize_detection(file_name, detections['prediction'], object_categories, threshold)